<a href="https://colab.research.google.com/github/AnnLivio/MySQL_Projects/blob/main/Baby_Names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baby Name Trend Analysis
View trends in baby name popularity based on US social security data from 1980-2010.
+ Libraries: SQLite, Pandas.
+ Origen: https://app.mavenanalytics.io/guided-projects/f71c0a2b-05f4-43fe-a80c-8f3f86964ccc
+ Guided Project from Maven Analytics for MySQL.

In [1]:
# Import libraries
import sqlite3
import csv
import pandas as pd

**Create cursor and conexion**

In [2]:
con = sqlite3.connect("baby_names.db")
cur = con.cursor()

In [4]:
# Load sql extension to make queries
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
%sql sqlite:///baby_names.db

In [6]:
# It contains the table description
bn = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/baby_names_db_data_dictionary.csv")
bn

,Table,Field,Description
0,names,State,State (abbreviation) where the babies were born
1,names,Gender,Gender of the babies at birth
2,names,Year,Year the babies were born
3,names,Name,Name given to the babies
4,names,Births,Number of babies given a name for a specific s...
5,regions,State,State (abbreviation) in the United States
6,regions,Region,Region of the United States that the state is ...


**Create tables Names and Regions**

In [7]:
# Create names table
%%sql CREATE TABLE names (
    State CHAR(2),
    Gender CHAR(1),
    Year INT,
    Name VARCHAR(45),
    Births INT);

 * sqlite:///baby_names.db
Done.


[]

In [35]:
# Create Region Table
%%sql CREATE TABLE regions (
      State CHAR(2),
      Region VARCHAR(45)
      );

 * sqlite:///baby_names.db
Done.


[]

In [36]:
# Insert values in regions table
%%sql
INSERT INTO regions VALUES ('AL', 'South'),
('AK', 'Pacific'),('AZ', 'Mountain'),('AR', 'South'),
('CA', 'Pacific'),('CO', 'Mountain'),('CT', 'New_England'),
('DC', 'Mid_Atlantic'),('DE', 'South'),('FL', 'South'),
('GA', 'South'),('HI', 'Pacific'),('ID', 'Mountain'),
('IL', 'Midwest'),('IN', 'Midwest'),('IA', 'Midwest'),
('KS', 'Midwest'),('KY', 'South'),('LA', 'South'),
('ME', 'New_England'),('MD', 'South'),('MA', 'New_England'),
('MN', 'Midwest'),('MS', 'South'),('MO', 'Midwest'),
('MT', 'Mountain'),('NE', 'Midwest'),('NV', 'Mountain'),
('NH', 'New England'),('NJ', 'Mid_Atlantic'),('NM', 'Mountain'),
('NY', 'Mid_Atlantic'),('NC', 'South'),('ND', 'Midwest'),
('OH', 'Midwest'),('OK', 'South'),('OR', 'Pacific'),
('PA', 'Mid_Atlantic'),('RI', 'New_England'),('SC', 'South'),
('SD', 'Midwest'),('TN', 'South'),('TX', 'South'),
('UT', 'Mountain'),('VT', 'New_England'),('VA', 'South'),
('WA', 'Pacific'),('WV', 'South'),('WI', 'Midwest'),
('WY', 'Mountain');

 * sqlite:///baby_names.db
50 rows affected.


[]

In [42]:
# Insert MI state
%%sql INSERT INTO regions VALUES ('MI', 'Midwest');

 * sqlite:///baby_names.db
1 rows affected.


[]

In [44]:
# Normalize New England
%%sql UPDATE regions SET Region = 'New_England' WHERE
    Region = 'New England';

 * sqlite:///baby_names.db
1 rows affected.


[]

**Import names data**

In [9]:
# import table CHEESE PRODUCTION
cols = ["State", "Gender", "Year", "Name", "Births"]
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/names_data.csv", header=None, names=cols)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2212361 entries, 0 to 2212360
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   State   object
 1   Gender  object
 2   Year    int64 
 3   Name    object
 4   Births  int64 
dtypes: int64(2), object(3)
memory usage: 84.4+ MB


In [11]:
#Divide the dataset because is too large
# 35,000 is the max for now, i don't know

# j = df Lenght
j = 2212360
i = 0

# While loop to append data by 35000 records each time
while j >= 35000:
  df_new = df.iloc[i:i+35000,]
  # Append df_new to table "names"
  df_new.to_sql("names", con, if_exists='append', index=False, method="multi")
  i += 35000
  j -= 35000

# Append the last records
df_new = df.iloc[i:2212361,]
df_new.to_sql("names", con, if_exists='append', index=False, method="multi")

7361

In [12]:
%sql SELECT COUNT(*) FROM names;

 * sqlite:///baby_names.db
Done.


COUNT(*)
2212361


In [46]:
del df_new
del df

In [13]:
#Show the table schema
%sql PRAGMA table_info("names")

 * sqlite:///baby_names.db
Done.


cid,name,type,notnull,dflt_value,pk
0,State,CHAR(2),0,None,0
1,Gender,CHAR(1),0,None,0
2,Year,INT,0,None,0
3,Name,VARCHAR(45),0,None,0
4,Births,INT,0,None,0


## Task 1:

Find the overall most popular girl and boy names and show how they have changed in popularity rankings over the years.

In [14]:
# Girls most popular names
%%sql
  SELECT Name, SUM(Births) as num_babies
  FROM names
  WHERE Gender = 'F'
  GROUP BY Name
  ORDER BY num_babies DESC
  Limit 5;

 * sqlite:///baby_names.db
Done.


Name,num_babies
Jessica,863121
Ashley,786945
Jennifer,652244
Sarah,621174
Amanda,607253


In [15]:
# Boys most popular names
%%sql
  SELECT Name, SUM(Births) as num_babies
  FROM names
  WHERE Gender = 'M'
  GROUP BY Name
  ORDER BY num_babies DESC
  Limit 5;

 * sqlite:///baby_names.db
Done.


Name,num_babies
Michael,1376418
Christopher,1118253
Matthew,1031984
Joshua,957518
Daniel,821281


### The most popular name over from 1980 to 2010 for girls is `Jessica` and for boys is `Michael`.

In [16]:
# Jessica changes in popularity ranking over the years
%%sql
  SELECT Year, Name, popularity
  FROM(
  WITH girl_names AS(
    SELECT Year, Name, SUM(Births) as num_babies
    FROM names
    WHERE Gender = 'F'
    GROUP BY Year, Name)
  SELECT Year, Name,
      ROW_NUMBER() OVER(PARTITION BY Year
                        ORDER BY num_babies DESC) as popularity
  FROM girl_names) girl_rank
  WHERE Name = 'Jessica';



 * sqlite:///baby_names.db
Done.


Year,Name,popularity
1980,Jessica,3
1981,Jessica,2
1982,Jessica,2
1983,Jessica,2
1984,Jessica,2
1985,Jessica,1
1986,Jessica,1
1987,Jessica,1
1988,Jessica,1
1989,Jessica,1


In [19]:
# Michael changes in popularity ranking over the years
%%sql
  SELECT Year, Name, popularity
  FROM(
  WITH boy_names AS(
    SELECT Year, Name, SUM(Births) as num_babies
    FROM names
    WHERE Gender = 'M'
    GROUP BY Year, Name)
  SELECT Year, Name,
      ROW_NUMBER() OVER(PARTITION BY Year
                        ORDER BY num_babies DESC) as popularity
  FROM boy_names) boy_rank
  WHERE Name = 'Michael';

 * sqlite:///baby_names.db
Done.


Year,Name,popularity
1980,Michael,1
1981,Michael,1
1982,Michael,1
1983,Michael,1
1984,Michael,1
1985,Michael,1
1986,Michael,1
1987,Michael,1
1988,Michael,1
1989,Michael,1


## Task 2
Find the names with the biggest jumps in popularity from the first year of the data set to the last year

In [20]:
%%sql
  WITH all_names AS(
    SELECT Year, Name, SUM(Births) as num_babies
    FROM names
    GROUP BY Year, Name),

-- Ranking name for 1980
  older AS(
    SELECT Year, Name,
                  ROW_NUMBER() OVER(PARTITION BY Year ORDER BY num_babies DESC) as popularity_o
            FROM all_names
            WHERE Year = 1980),
-- Ranking name for 2009
  recent AS(
    SELECT Year, Name,
      ROW_NUMBER() OVER(PARTITION BY Year ORDER BY num_babies DESC) as popularity_p
      FROM all_names
      WHERE Year = 2009)

  SELECT Name, popularity_o as rank_1980, popularity_p as rank_2009, (popularity_p - popularity_o) as jump
  FROM (SELECT o.Name, o.popularity_o, r.popularity_p
        FROM older o
        INNER JOIN recent r
        ON o.Name = r.Name) rank
  ORDER BY jump
  LIMIT 10;


 * sqlite:///baby_names.db
Done.


Name,rank_1980,rank_2009,jump
Skylar,5667,313,-5354
Rylan,5601,320,-5281
Rowan,5595,445,-5150
Colton,5103,149,-4954
Aidan,4995,109,-4886
Lexi,5415,537,-4878
Macy,5440,571,-4869
Kyler,5370,511,-4859
Norah,5519,672,-4847
Tate,5707,901,-4806


# Compare popularity across decades

## Task 1
For each year, return the 3 most popular girl names and 3 most popular boy names

In [30]:
%%sql
  WITH all_names AS(
            SELECT Year, Name, Gender, SUM(Births) as num_babies
            FROM names
            GROUP BY Year, Name, Gender
  ),
  ranking_names AS(
            SELECT Year, Name, Gender,
              ROW_NUMBER() OVER(PARTITION BY Year, Gender ORDER BY num_babies DESC) as popularity_g
            FROM all_names
  )

  SELECT * FROM ranking_names
  WHERE popularity_g <= 3
  ORDER BY Year, Gender, popularity_g;

 * sqlite:///baby_names.db
Done.


Year,Name,Gender,popularity_g
1980,Jennifer,F,1
1980,Amanda,F,2
1980,Jessica,F,3
1980,Michael,M,1
1980,Christopher,M,2
1980,Jason,M,3
1981,Jennifer,F,1
1981,Jessica,F,2
1981,Amanda,F,3
1981,Michael,M,1


## Task 2
For each decade, return the 3 most popular girl names and 3 most popular boy names

In [34]:
%%sql
  WITH names_by_decade AS(
            SELECT
              (CASE when Year between 1980 and 1989 then '1980-1989'
                    when Year between 1990 and 1999 then '1990-1999'
                    when Year between 2000 and 2009 then '2000-2009'
                    else 'None'
              END) as Decade,
              Name, Gender, SUM(Births) as num_babies
            FROM names
            GROUP BY Decade, Name, Gender
  ),
  ranking_names AS(
            SELECT Decade, Name, Gender,
              ROW_NUMBER() OVER(PARTITION BY decade, Gender ORDER BY num_babies DESC) as Popularity
            FROM names_by_decade
  )

SELECT Decade, Name, Gender, Popularity
FROM ranking_names
WHERE popularity <= 3
GROUP BY Decade, Gender, Popularity;

 * sqlite:///baby_names.db
Done.


Decade,Name,Gender,Popularity
1980-1989,Jessica,F,1
1980-1989,Jennifer,F,2
1980-1989,Amanda,F,3
1980-1989,Michael,M,1
1980-1989,Christopher,M,2
1980-1989,Matthew,M,3
1990-1999,Jessica,F,1
1990-1999,Ashley,F,2
1990-1999,Emily,F,3
1990-1999,Michael,M,1


# Compare popularity across regions
## Task 1
Return the number of babies born in each of the six regions (NOTE: The state of MI should be in the Midwest region)

In [45]:
%%sql
  SELECT r.Region, SUM(n.Births) as num_babies
  FROM names n
  INNER JOIN regions r
  ON n.State = r.State
  GROUP BY r.Region
  ORDER BY num_babies DESC;

 * sqlite:///baby_names.db
Done.


Region,num_babies
South,34219920
Midwest,22676130
Pacific,17540716
Mid_Atlantic,13742667
Mountain,6282217
New_England,4269213


## Task 2
Return the 3 most popular girl names and 3 most popular boy names within each region

In [47]:
%%sql
  WITH names_by_region AS(
            SELECT r.Region, n.Name, n.Gender, SUM(n.Births) as num_babies
            FROM names n
            INNER JOIN regions r
            ON n.State = r.State
            GROUP BY r.Region, n.Name, n.Gender
  ),
  ranking_names AS(
            SELECT Region, Name, Gender,
              ROW_NUMBER() OVER(PARTITION BY Region, Gender ORDER BY num_babies DESC) as Popularity
            FROM names_by_region
  )

SELECT Region, Name, Gender, Popularity
FROM ranking_names
WHERE popularity <= 3
GROUP BY Region, Gender, Popularity;

 * sqlite:///baby_names.db
Done.


Region,Name,Gender,Popularity
Mid_Atlantic,Jessica,F,1
Mid_Atlantic,Ashley,F,2
Mid_Atlantic,Jennifer,F,3
Mid_Atlantic,Michael,M,1
Mid_Atlantic,Matthew,M,2
Mid_Atlantic,Christopher,M,3
Midwest,Jessica,F,1
Midwest,Ashley,F,2
Midwest,Sarah,F,3
Midwest,Michael,M,1


# Explore unique names in the dataset
## Task 1
Find the 10 most popular androgynous names (names given to both females and males)

In [52]:
%%sql
  SELECT Name, COUNT(DISTINCT(Gender)) as count_g, SUM(Births) as num_babies
  FROM names
  GROUP BY Name
  HAVING count_g = 2
  ORDER BY num_babies DESC
  LIMIT 10;

 * sqlite:///baby_names.db
Done.


Name,count_g,num_babies
Michael,2,1382856
Christopher,2,1122213
Matthew,2,1034494
Joshua,2,960170
Jessica,2,865046
Daniel,2,824208
David,2,819479
Ashley,2,792865
James,2,766789
Andrew,2,761824


## Task 2
Find the length of the shortest and longest names, and identify the most popular short names (those with the fewest characters) and long names (those with the most characters)

In [55]:
%%sql SELECT MIN(LENGTH(Name)), MAX(LENGTH(Name)) FROM names;

 * sqlite:///baby_names.db
Done.


MIN(LENGTH(Name)),MAX(LENGTH(Name))
2,15


In [85]:
%%sql
  WITH length_names AS(
    SELECT Name, length(Name) as length_name, SUM(Births) as num_babies
    FROM names
    GROUP BY Name, length_name
    HAVING LENGTH(Name) IN (2,15)
  )
  SELECT * FROM length_names
  GROUP BY length_name
  HAVING MAX(num_babies);


 * sqlite:///baby_names.db
Done.


Name,length_name,num_babies
Ty,2,29205
Franciscojavier,15,52


## Task 3
The founder of Maven Analytics is named Chris. Find the state with the highest percent of babies named "Chris"

In [117]:
%%sql
  WITH state_total_names AS(
    SELECT State, SUM(Births) as num_babies FROM names
    GROUP BY State
  ),
  state_chris_total as (
    SELECT State, SUM(Births) as num_chris FROM names
    WHERE Name = 'Chris'
    GROUP BY State
  )

  SELECT sc.State, ROUND(CAST(sc.num_chris AS REAL) / st.num_babies * 100,4) as percent_chris
  FROM state_chris_total sc
  INNER JOIN state_total_names st
  ON sc.State = st.State;

 * sqlite:///baby_names.db
Done.


State,percent_chris
AL,0.0143
AR,0.0156
AZ,0.0186
CA,0.0268
CO,0.015
CT,0.0136
DC,0.0115
DE,0.0041
FL,0.0209
GA,0.0199


In [118]:
con.close()